The below code bases upon the assumption that blick_sitegetter has been executed. The below code will get all data from the defined Blick sites:


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
import csv

In [2]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

In [3]:
kommentare = []

class Kommentar:
    def __init__(self, url, datum, text):
        self.url = url
        self.datum = datum
        self.text = text

In [4]:
start_seite = "https://www.blick.ch/"
driver.get(start_seite)

cookies = pickle.load(open("blick_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [5]:
with open("blick_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)

In [6]:
def inhalter():
   
    klicker = (comm_count // 5) - 2

    for anzahl in range(klicker):
        try:
            x = driver.find_element_by_xpath("//button[contains(.,'Weitere Kommentare anzeigen')]")
            time.sleep(1)
            x.click()
        except NoSuchElementException or StaleElementReferenceException:
            print("Final one")
            break    
            
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    comment = soup.find_all(class_=re.compile("ItemWrapper__Wrapper-c5w9wz-0.cRSEgg.commenting-item.comment-item"))

    for i in comment:
        texty = i.find(class_=re.compile("ItemTextContent"))
        texty = texty.get_text()
        text = texty.replace("\n", "")


        datum = i.find(class_=re.compile("ItemHeaderDatePublished"))
        datum = datum.get_text()

        crawled_kom = Kommentar(page, datum, text)
        kommentare.append(crawled_kom)

In [15]:
for page in ueb_pages:
    driver.get(page)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    try:
        comm_count = soup.find(class_=re.compile("button headline-button headline-button--align-right"))
        comm_count = comm_count.get_text().strip()
        comm_count = comm_count.split(" ")
        if len(comm_count) == 1:
            print("No comments yet")
            pass
        else:
            comm_count = comm_count[0]
            comm_count = comm_count[:3]
            comm_count = int(comm_count)
            inhalter()
    except (AttributeError, NoSuchElementException, TimeoutException, StaleElementReferenceException):
            print("Error - we go on")

Final one
Final one
Final one
Final one
No comments yet
No comments yet
No comments yet
No comments yet
No comments yet
Final one
No comments yet
Final one
No comments yet
No comments yet
Final one
Final one
Final one
Final one
No comments yet
Final one
No comments yet
Final one
Final one
Final one
Final one
No comments yet
No comments yet
Final one
No comments yet
Final one
Final one
Final one
No comments yet
Final one
No comments yet
Final one
Final one
Final one
No such element - we go on
No comments yet
Final one
No comments yet
No comments yet
Final one
No comments yet
No comments yet
Final one
No such element - we go on
No comments yet
Final one
No comments yet
No comments yet
No comments yet
No comments yet
Final one
No comments yet
No comments yet
Final one
No comments yet
No comments yet
Final one
Final one
No comments yet
Final one
No comments yet
No comments yet
Final one
No comments yet
Final one
No comments yet
Final one
No comments yet
No comments yet
No comments yet
Fina

TimeoutException: Message: 


In [16]:
with open('blick_kom.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter_kom = csv.writer(csvfile, delimiter='|', quotechar='"')
                                
    for element in kommentare:
        blogwriter_kom.writerow( [element.url, element.datum, element.text] )

    

# Quick fix
In case one needs to interrupt crawling and wants to continue later where the crawler stopped, run this cell and update cell 7, first line from: <br>
*"for page in ueb_pages:"* <br>
to : <br>
*"for page in ueb_pages[go_on:]:"*

This method also works in case of uncatched errors or interrupted kernels, provided kernel has not been restarted yet. In case kernel will be restarted, please note down the value of go_on or pickle dump the value beforehand as the value of go_on will not be stored automatically.


In [ ]:
latest = kommentare[-1].url        ##get the url from the last object that has been scraped
go_on = ueb_pages.index(latest)   ##find this url in the list that is crawled through and extract its index
print(go_on)